<h1 align='center'>LBSN Project</h1>

<h4 align='center'>Author : Pierre ADEIKALAM, Guangyue CHEN, Kevin XU</h4>

# Table of content

[1. Introduction](#introduction)<br>
[2. Processing of the Dataset](#processdata)<br>
[3. Geographical Computations](#geocomp)<br>
[4. Social computations](#socialcomp)<br>
[5. Generate & Test Recommendations](#genetest)<br>

# Some imports

In [10]:
colab = True
if colab:
    !pip install scikit-surprise
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1scymDaABRjt8IWP4hlD6uLdg0nVXzi3E', dest_path='./gowalla.zip')

    from zipfile import ZipFile
    with ZipFile('gowalla.zip', 'r') as zipObj:
        # Extract all the contents of zip file in current directory
        zipObj.extractall()

In [0]:
# import
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise import AlgoBase
from surprise import PredictionImpossible
from surprise.model_selection import cross_validate

from tqdm.notebook import tqdm

<a id='intro'></a>
# 1. Introduction

The purpose of this project is to implement a location-based recommender system from regular location-based social network (LBSN) data. Usually LBSN data give precise information regarding the locations visited by the users. These data also reveal who are the friends of the users. These two information (geographical + social) can be exploited into a standard collaborative filtering approach. At the end of this project we have all the building blocks of the model iGSLR, a geo-social location recommendation model.

<a id='processdata'></a>
# 2. Processing of the Dataset

In [0]:
min_num_checkins = 10 # 5

## 2.1 Import data

In [0]:
checkins = pd.read_csv('gowalla/gowalla_checkins.csv')
locations = pd.read_csv('gowalla/gowalla_locations.csv')
friendship = pd.read_csv('gowalla/gowalla_friendship.csv')
userinfo = pd.read_csv('gowalla/gowalla_userinfo.csv')

In [14]:
# Reduce original size by k
k = 1
userinfo = userinfo.iloc[::k]

for db in [checkins, friendship, locations, userinfo]:
    print(db.shape)

(36001959, 2)
(4418339, 2)
(2724891, 5)
(407533, 3)


In [15]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
display('checkins.head(3)', 'locations.head(3)', 'friendship.head(3)', 'userinfo.head(3)')

checkins.head(3)
   userid  placeid
0    1338   482954
1    1338   580963
2    1338   365256

locations.head(3)
     id        lng        lat  checkins_count  users_count
0  8904 -94.607499  39.052318             114           21
1  8932 -97.254356  32.927662              67           48
2  8936 -94.591995  39.053318              75           46

friendship.head(3)
   userid1  userid2
0        1    63488
1        1        2
2        1        3

userinfo.head(3)
   id  friends_count  checkin_num
0   1            372         1766
1   2            775         2892
2   3            100         3021

## 2.2 Creation of additional DataFrames

In [16]:
# Filter users who have less than min_num_checkins check-ins in total
mask_filter_user = userinfo.set_index('id')['checkin_num'] > min_num_checkins
userinfo = userinfo.set_index('id')[mask_filter_user].reset_index()
userinfo

,id,friends_count,checkin_num
0,1,372,1766
1,2,775,2892
2,3,100,3021
3,4,179,1325
4,5,525,3215
...,...,...,...
199283,2688261,2,12
199284,2688369,7,16
199285,2688642,1,11
199286,2688873,1,13


In [17]:
# Associate to each user the list of his friends
friendship = friendship[friendship['userid1'].isin(userinfo['id']) & friendship['userid2'].isin(userinfo['id'])] # Keep only users in userinfo
df_user_friends = friendship.groupby('userid1')['userid2'].apply(list).reset_index()
df_user_friends.columns = ['userid', 'friendslist']
df_user_friends.head(10)

,userid,friendslist
0,1,"[63488, 2, 3, 4, 5, 6, 7, 12, 2065, 18, 24, 25..."
1,2,"[131140, 1, 55298, 3, 4, 5, 6, 7, 11, 12, 9327..."
2,3,"[1, 2, 4, 5, 6, 7, 12, 18, 2315532, 29, 309765..."
3,4,"[1, 2, 3, 5, 6, 7, 12, 65553, 18, 24, 25, 29, ..."
4,5,"[1, 2, 3, 4, 6, 7, 11, 12, 18, 24, 25, 29, 33,..."
5,6,"[1765005, 1, 2, 3, 4, 5, 7, 12, 381453, 18, 88..."
6,7,"[1, 2, 3, 4, 5, 6, 3592, 11, 12, 5646, 130576,..."
7,11,"[2, 153899, 5, 7, 2696, 208011, 253347, 29329,..."
8,12,"[1, 2, 3, 4, 5, 6, 7, 2139144, 15959, 3087, 12..."
9,15,"[256, 578, 2819, 136860, 52810, 59307, 2094, 2..."


In [18]:
# Associate to each user the list of locations he visited
df_user_locations = checkins.groupby('userid')['placeid'].apply(lambda x: list(set(x))).reset_index(name='locationslist')
df_user_locations = userinfo.merge(df_user_locations, left_on='id', right_on='userid', how='left')[['userid', 'locationslist']]
df_user_locations

,userid,locationslist
0,1,"[34817, 88067, 681993, 40971, 16397, 12305, 10..."
1,2,"[88066, 20486, 182280, 90121, 681993, 40971, 1..."
2,3,"[10244, 663558, 10246, 30728, 702472, 7047178,..."
3,4,"[32769, 10244, 663558, 681993, 32778, 40971, 1..."
4,5,"[88066, 10244, 22535, 681993, 40971, 24587, 18..."
...,...,...
199283,2688261,"[6897828, 5250220, 7534156, 6195919, 874641, 7..."
199284,2688369,"[7541633, 7531746, 7538150, 7544615, 7538471, ..."
199285,2688642,"[7539298, 7534250, 317292, 6584749, 4902767]"
199286,2688873,"[7548381, 1466807, 1428136, 7538414, 414030, 7..."


In [19]:
# Visit frequency and transformation
df_frequencies = checkins.groupby(['userid', 'placeid'])['placeid'].count().reset_index(name ='frequency')
df_frequencies = userinfo.merge(df_frequencies, left_on='id', right_on='userid', how='left')[['userid', 'placeid', 'frequency']]
df_frequencies

,userid,placeid,frequency
0,1,8904,72
1,1,8938,11
2,1,8947,12
3,1,8956,1
4,1,8957,1
...,...,...,...
19096080,2688969,7450619,9
19096081,2688969,7466667,1
19096082,2688969,7538565,4
19096083,2688969,7543852,1


In [0]:
def frequency_normalization(x, fmin, fmax):
    """ 
    Normalize the frequency x into the range [0, 10].
    
    Parameters:
    ----------
    x : float
        The value to be normalized.
    fmin : float
        The minimum visit frequency of the dataset.
    fmax : float
        The maximum visit frequency of the dataset.
    
    Returns:
    -------
    x_new : float
        The normalized value of x.
    """
    return 10 * np.tanh(10 * (x - fmin) / (fmax - fmin))

fmin = df_frequencies['frequency'].min()
fmax = df_frequencies['frequency'].max()

df_frequencies['frequency'] = df_frequencies['frequency'].apply(frequency_normalization, fmin=fmin, fmax=fmax)

In [0]:
reader = Reader(rating_scale=(0, 10))
df_frequencies_new = Dataset.load_from_df(df_frequencies, reader)

# Split data
X_train, X_test = train_test_split(
    df_frequencies_new, test_size=0.25, random_state=42
)

<a id='geocomp'></a>
# 3. Geographical Computations

In [0]:
 def normal_kernel(x):
    """
    Computes the normal kernel function with fixed bandwith at a given point x.

    Parameters:
    ----------
    x : float
        Point to evaluate
    
    Returns:
    -------
    y : float
    """
    return np.exp(-x**2/2)/np.sqrt(2*np.pi)

def density(x, distances):
    """
    Computes the density at point x of a specific user given an array of distances between the locations
    the user has visited.

    Parameters: 
    ----------
    x : float
        Point to evaluate.

    distances : numpy array of distances.
    
    Returns:
    ----------
    density : float
        The density of x.
    """

    sigma = distances.std() # standard deviation of the distances
    n = len(distances)**0.5         # d is of length n**2
    h = 1.06 * sigma * (n**(-1/5))

    normalizing_constant = (len(distances) * h)
    density = normal_kernel((x - distances)/h).sum()
    return density / normalizing_constant

def distance_between_locations(location1, location2):
    """
    Computes the l2 distance between two locations given their geographical coordinates.

    Parameters:
    ----------
    location1, location2 : tuple (x, y) of geographical coordinates.
    
    Returns:
    ----------
    distance : float
    """
    dx = (location1[0] - location2[0])**2
    dy = (location1[1] - location2[1])**2
    distance = np.sqrt(dx + dy)

    return distance

def geo_proba(locations, new_location):
    """
    Geographical likelihood that a user will visit a new location given the locations they
    have visited.

    Parameters:
    ----------
    locations: list of locations the user has visited. Each entry of the list is a tuple giving
             the geographical coordinates of the location.

    new_location : a tuple of geographical coordinates of a new location.
    """
    # print("Number of location visited by the current user :", len(locations))

    # Sample the visited locations
    if len(locations) > max_visited_locations:
        locations = random.sample(list(locations), max_visited_locations)

    densities = np.zeros(len(locations))
    distances = [distance_between_locations(location1, location2) for location1 in locations for location2 in locations]
    distances = np.array(distances)

    for i, visited_location in enumerate(locations):
        distance_to_new = distance_between_locations(visited_location, new_location)
        densities[i] = density(distance_to_new, distances)

    return densities.mean()

In [0]:
# Let's test our functions
user = 1

# Get the list of locations the user has visited
visited_locations_df = locations.set_index('id').loc[df_user_locations.query('userid == @user').values[0, 1]][['lng', 'lat']]
visited_locations_list = visited_locations_df.values

# Each entry of the list is a tuple giving the geographical coordinates of the location.

In [24]:
max_visited_locations = 100

geo_proba(visited_locations_list, visited_locations_list[1])

0.023264398221923072

## Number of visited locations by users

In [25]:
locations_count_by_user = df_frequencies.groupby('userid')['placeid'].count()
locations_count_by_user.nlargest(10)

userid
213489    34591
84414     30569
114774    28923
76390     25146
30603     24854
28509     24291
9298      20953
179386    20503
46646     18613
470092    17661
Name: placeid, dtype: int64

In [26]:
np.sum(locations_count_by_user > 100)/locations_count_by_user.shape[0]

0.19915398819798483

> In order to be able to compute the probability for some users, we have to reduce the number of visited locations. Indeed, we notice that a large amount users have visited more than 100 locations. Thus the computation of the distance between each location can require a very long computation time.
> 
> That's why, we sample a fixed number of locations for the users which raise this issue.

<a id='socialcomp'></a>
# 4. Social computations

In [0]:
def social_similarity(u, v):
    """ 
    Compute social similarity score with the Jaccard coefficient.
    
    Parameters:
    ----------
    u : int
        An raw user id.
    v : int
        An raw user id.
    
    Returns:
    -------
    score : float
        The social similarity score between user u and v.
    """
    try:
        F_u = set(df_user_friends.query('userid == @u')['friendslist'].tolist()[0])
        F_v = set(df_user_friends.query('userid == @v')['friendslist'].tolist()[0])
    except:
        print('Not valid user id')
        return -1
    return len(F_u.intersection(F_v)) / len(F_u.union(F_v))

In [0]:
# Compute r_hat prediction
def social_rating(i, j, trainset):
    """ 
    Compute the frequency of user i visiting location j with social information.
    
    Parameters:
    ----------
    i : int
        An inner user id.
    j : int
        A inner location id.
    trainset : DataFrame
        The training set
    Returns:
    -------
    score : float
        The frequency
    """
    raw_user_id = trainset.to_raw_uid(i)

    F_i = df_user_friends.query('userid == @raw_user_id')['friendslist'].tolist()[0]
    F_i = [trainset.to_inner_uid(friend_id) for friend_id in F_i]

    location_frequencies = dict(trainset.ir[j])
    F_visited_location = set(F_i).intersection(set(location_frequencies.keys()))

    num = 0
    den = 0
    score = 0
    
    for friend_id in F_visited_location:
        # Compute social similarity between the two users
        sim = social_similarity(raw_user_id, trainset.to_raw_uid(friend_id))
    
        # Get the friend's location frequency
        r = location_frequencies[friend_id]
        num += r * sim
        den += sim

    if den != 0:
        score = num/den
    return score

In [0]:
# Compute probability p_hat
def social_probability(i, j, trainset):
    """ 
    Compute the probability of user i visited location j with social information.
    
    Parameters:
    ----------
    i : int
        A raw user id.
    j : int
        A raw location id.
    trainset : DataFrame
        The training set
    Returns:
    -------
    p_hat : float
        The probability
    """
    inner_user_id = trainset.to_inner_uid(i)
    inner_location_id = trainset.to_inner_iid(j)

    try:
        F_i = df_user_friends.query('userid == @i')['friendslist'].tolist()[0]
    except:
        return 0  
    F_i = [trainset.to_inner_uid(friend_id) for friend_id in F_i]
    user_locations = set(dict(trainset.ur[inner_user_id]).keys())
    
    location_ids = set()
    for friend_i in F_i:
        location_ids.update(set(dict(trainset.ur[friend_i]).keys()))

    location_ids = location_ids.difference(user_locations)
    
    # Sample locations visited by friends
    if len(location_ids) > max_number_locations:
        location_ids = random.sample(location_ids, max_number_locations)
        
    max_r = -1
    # print("------- Computation of max r_hat -------")
    for location_id in location_ids: # tqdm(location_ids): 
        r_hat2 =  social_rating(inner_user_id, location_id, trainset)
        if max_r < r_hat2:
            max_r = r_hat2
    # print(max_r)

    r_hat = social_rating(inner_user_id, inner_location_id, trainset)
    
    p_hat = 0
    if max_r != 0:
        p_hat = r_hat / max_r
    return p_hat

In [30]:
max_number_locations = 100

i = 1	
j = 8904
social_probability(i, j, X_train)

0.05323633101101231

## Number of locations visited by user's friends

> The normalization term $max_{l_j \in L-L_i}\{\hat{r}_{i,j}\}$ requires a very long computation time because we have to go through all the locations in the dataset (about 2,724,891 locations). 
> 
> But since in the equation of $\hat{r}_{i,j}$, the rate $\hat{r}_{k,j}$ is equal to $0$ when the friend $k$ did not visit the location $i$, we can include in $L$ only the places that have been visited at least once by any friend of user $i$.  
>
> Besides, some users still have too many locations. That's why, we sample these locations as in the previous section.

<a id='genetest'></a>
# 5. Generate & Test Recommendations

In [0]:
# Geographical Social Location Recommendation Class
class GSLR(AlgoBase):

    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        return self

    def estimate(self, i, j):
        """ 
        Generate a recommendation score for a given user and a given location
        
        Parameters:
        ----------
        i : int
            An inner user id.
        j : int
            An inner location id.
        
        Returns:
        -------
        score : float
        """
        try:
            self.trainset.knows_user(i) # self.trainset.knows_user(self.trainset.to_inner_uid(i))
            self.trainset.knows_item(j) # self.trainset.knows_item(self.trainset.to_inner_iid(j))
            raw_user_id = self.trainset.to_raw_uid(i)
            raw_location_id = self.trainset.to_raw_iid(j)
        except:
            # raise PredictionImpossible('User and/or item is unkown.')
            print('User and/or item is unknown.')
            return 0
      
        # Geographical computations
        try:
            ## Get the list of locations the user has visited
            visited_locations_list = locations.set_index('id').loc[df_user_locations.query('userid == @raw_user_id').values[0, 1]][['lng', 'lat']].values
            new_location_coord = locations.set_index('id').loc[raw_location_id, ['lng', 'lat']].values
            geo_prob = geo_proba(visited_locations_list, new_location_coord)
            
            # Social computations
            social_prob = social_probability(raw_user_id, raw_location_id, self.trainset)
        except:
            print("Unvalid visited location")
            return 0
        
        
        return (social_prob + geo_prob) / 2

In [45]:
max_visited_locations = 100
max_number_locations = 100

recommender = GSLR()
recommender.fit(X_train)

i = X_train.to_inner_uid(1)
j = X_train.to_inner_iid(8904) 
recommender.estimate(i, j)

0.015131423250069017

## Compute Recall and Precision

In [0]:
from collections import defaultdict
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """ 
    Compute precision and recall at k metrics for each user

    Parameters:
    ----------
    predictions : A Prediction object 
    k : int
        The top-k highest scores.
    threshold : float
        Threshold for an item to be considered relevant. 
    Returns:
    -------
    precisions, recalls : floats
    """
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [48]:
# Generate recommendations for each user in the test set and get the precision@K and recall@K
max_visited_locations = 100
max_number_locations = 100
testset_size = 200

k = 5 # 5, 10, 15

# Build model
recommender = GSLR()
recommender.fit(X_train)

# Sample the testset
testset_sampled = random.sample(X_test, testset_size)

# Predict on test set
predictions = recommender.test(testset_sampled)
precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=2)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
User and/or item is unknown.
Unvalid visited location
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
User and/or item is unknown.
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvalid visited location
Unvali

> We observe that a large amount of users or items are unknown because they are not in the training set. For instance, out of 100 samples from the testset, there are about 7 scores that cannot be computed. However, the computation still takes a long time even with limited number of locations.
> 
> The parameters we have chosen do not give us realistic results. Indeed, the computed precision@K and recall@K are fairly high which does not seem relevant. Due to limits of computation and time, we were not able to let the algorithm run on the entire dataset in order to get more reliable results.

In [0]:
max_visited_locations = 100
max_number_locations = 100
testset_size = 50

k = 5 # [5, 10, 15]

kf = KFold(n_splits=3)
recommender = GSLR()

for trainset, testset in kf.split(df_frequencies_new):
    testset_sampled = random.sample(testset, testset_size)

    recommender.fit(trainset)
    predictions = recommender.test(testset_sampled)
    precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=1)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
User and/or item is unknown.
1.0
1.0
